- Identificar protocolos de comunicacion mas usados.
- Ver cuales son los mas seguros.
- A que tipo de ataque son mas propensos estos protocolos.

In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


UNSW_NB15 e IOT_TON (Network) datasets: https://research.unsw.edu.au/projects/unsw-nb15-dataset


In [69]:
network1= pd.read_csv("data/Network_dataset_1.csv")
network1.head()

C:\Users\nivek\AppData\Local\Temp\ipykernel_13912\110905391.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  network1= pd.read_csv("data/Network_dataset_1.csv")


,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1554198358,3.122.49.24,1883,192.168.1.152,52976,tcp,-,80549.530260,1762852,41933215,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
1,1554198358,192.168.1.79,47260,192.168.1.255,15600,udp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
2,1554198359,192.168.1.152,1880,192.168.1.152,51782,tcp,-,0.000000,0,0,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
3,1554198359,192.168.1.152,34296,192.168.1.152,10502,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
4,1554198362,192.168.1.152,46608,192.168.1.190,53,udp,dns,0.000549,0,298,...,0,0,-,-,-,bad_UDP_checksum,-,F,0,normal


In [70]:
network21= pd.read_csv("data/Network_dataset_21.csv")
network21.head()

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1556351863,192.168.1.35,45133,192.168.1.1,53,udp,dns,0.000008,80,112,...,0,0,-,-,-,-,-,-,1,xss
1,1556351863,192.168.1.35,45606,176.28.50.165,80,tcp,http,1.208269,380,724,...,0,0,-,-,-,-,-,-,1,xss
2,1556351863,192.168.1.35,50797,192.168.1.1,53,udp,dns,0.008457,80,112,...,0,0,-,-,-,-,-,-,1,xss
3,1556351863,192.168.1.35,37234,52.28.231.150,80,tcp,http,1.022216,2074,1211,...,0,0,-,-,-,-,-,-,1,xss
4,1556351863,192.168.1.35,37222,52.28.231.150,80,tcp,http,1.071203,1816,1125,...,0,0,-,-,-,-,-,-,1,xss


In [71]:
unsw= pd.read_csv("data/UNSW_NB15_training-set.csv")
unsw.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0


# Transformaciones:

- Seleccionamos las categorias que vamos a usar:
  - Network (proto, service, duration, src_bytes, dst_bytes, conn_state, type, label)
  - UNSW (proto, service, dur, sbytes, dbytes, state, attack_cat, label)
- Renombrar las categorias.
- Estandarizar el estado de las conexiones.
- Unificar valores de tipo de ataque.
- Porcentaje de ataques segun el protocolo.
- Porcentaje de ataques segun el servicio.

# Resultados:

- Ranking de ataque por porcentaje de protocolo.
- Ranking de ataque por porcentaje de servicio.

In [72]:
network_columns = ["proto", "service", "duration", "src_bytes", "dst_bytes", "conn_state", "type", "label"]
unsw_columns = ["proto", "service", "dur", "sbytes", "dbytes", "state", "attack_cat", "label"]

In [73]:
network1 = network1[network_columns]
network1.rename(columns={"conn_state": "connection"}, inplace=True)
network1.head()

,proto,service,duration,src_bytes,dst_bytes,connection,type,label
0,tcp,-,80549.530260,1762852,41933215,OTH,normal,0
1,udp,-,0.000000,0,0,S0,normal,0
2,tcp,-,0.000000,0,0,OTH,normal,0
3,tcp,-,0.000000,0,0,OTH,normal,0
4,udp,dns,0.000549,0,298,SHR,normal,0


In [74]:
network21 = network21[network_columns]
network21.rename(columns={"conn_state": "connection"}, inplace=True)
network21.head()

,proto,service,duration,src_bytes,dst_bytes,connection,type,label
0,udp,dns,0.000008,80,112,SF,xss,1
1,tcp,http,1.208269,380,724,SF,xss,1
2,udp,dns,0.008457,80,112,SF,xss,1
3,tcp,http,1.022216,2074,1211,SF,xss,1
4,tcp,http,1.071203,1816,1125,SF,xss,1


In [75]:
unsw = unsw[unsw_columns]
unsw.rename(columns={
    "dur": "duration",
    "sbytes": "src_bytes",
    "dbytes": "dst_bytes",
    "state": "connection",
    "attack_cat": "type",
}, inplace=True)
unsw.head()

,proto,service,duration,src_bytes,dst_bytes,connection,type,label
0,tcp,-,0.121478,258,172,FIN,Normal,0
1,tcp,-,0.649902,734,42014,FIN,Normal,0
2,tcp,-,1.623129,364,13186,FIN,Normal,0
3,tcp,ftp,1.681642,628,770,FIN,Normal,0
4,tcp,-,0.449454,534,268,FIN,Normal,0


In [76]:
network1["connection"].value_counts()

connection
S0        635513
REJ       145597
OTH       132321
RSTOS0     48674
SHR        21159
SF         12406
RSTO        3325
S1           498
RSTRH        211
RSTR         176
S3            50
SH            45
S2            25
Name: count, dtype: int64

In [77]:
network21["connection"].value_counts()

connection
SF        964885
S0          8926
S1          8916
S2          7437
S3          5959
SHR         1610
SH          1176
OTH          973
REJ           84
RSTR          23
RSTO           8
RSTRH          2
RSTOS0         1
Name: count, dtype: int64

In [78]:
unsw["connection"].value_counts()

connection
INT    82275
FIN    77825
CON    13152
REQ     1991
RST       83
ECO       12
PAR        1
URN        1
no         1
Name: count, dtype: int64

In [79]:
connection_mapping = {
    "SF": "Finished",          # Connection successfully established and completed
    "S0": "Interrupted",       # SYN sent but no response received
    "S1": "Interrupted",       # Connection established but no data sent
    "S2": "Interrupted",       # Connection initiated but not completed
    "S3": "Interrupted",       # Connection partially established
    "OTH": "Other",            # Other
    "SH": "Connecting",        # SYN received, waiting for ACK
    "SHR": "Connecting",       # Shared handshake state
    "RSTO": "Reset",           # Reset after connection was open
    "RSTR": "Reset",           # Connection reset
    "RSTOS0": "Reset",         # Reset after SYN sent
    "RSTRH": "Reset",          # Reset with retransmission
    "REJ": "Interrupted",      # Connection rejected by the destination
    "FIN": "Finished",         # Connection gracefully terminated
    "INT": "Interrupted",      # Interrupted connection
    "CON": "Connecting",       # Connection currently active
    "ECO": "Request",          # Echo request (e.g., ICMP ping)
    "REQ": "Request",          # Request-based connection (e.g., HTTP, DNS)
    "RST": "Reset",            # Reset connection
    "PAR": "Other",            # Undefined or parameter-related connection
    "URN": "Other",            # Undefined or urgent connection
    "no": "Other"              # Unknown connection
}

network1["connection"] = network1["connection"].map(connection_mapping)
network21["connection"] = network21["connection"].map(connection_mapping)
unsw["connection"] = unsw["connection"].map(connection_mapping)


In [80]:
type_mapping = {
    "xss": "Cross-Site Scripting",
    "normal": "Normal",
    "scanning": "Scanning",
}

network1["type"] = network1["type"].map(type_mapping)
network21["type"] = network21["type"].map(type_mapping)

In [81]:
# Combine the datasets
data = pd.concat([network1, network21, unsw], ignore_index=True)

In [82]:
data["connection"].value_counts()

connection
Finished       1055116
Interrupted     895280
Other           133297
Reset            52503
Connecting       37142
Request           2003
Name: count, dtype: int64

In [83]:
data["type"].value_counts()

type
Cross-Site Scripting    985593
Scanning                791321
Normal                  279086
Generic                  40000
Exploits                 33393
Fuzzers                  18184
DoS                      12264
Reconnaissance           10491
Analysis                  2000
Backdoor                  1746
Shellcode                 1133
Worms                      130
Name: count, dtype: int64

In [84]:
data

,proto,service,duration,src_bytes,dst_bytes,connection,type,label
0,tcp,-,80549.530260,1762852,41933215,Other,Normal,0
1,udp,-,0.000000,0,0,Interrupted,Normal,0
2,tcp,-,0.000000,0,0,Other,Normal,0
3,tcp,-,0.000000,0,0,Other,Normal,0
4,udp,dns,0.000549,0,298,Connecting,Normal,0
...,...,...,...,...,...,...,...,...
2175336,udp,dns,0.000009,114,0,Interrupted,Generic,1
2175337,tcp,-,0.505762,620,354,Finished,Shellcode,1
2175338,udp,dns,0.000009,114,0,Interrupted,Generic,1
2175339,udp,dns,0.000009,114,0,Interrupted,Generic,1
